# Setup

In [1]:
# !pip install -qU gradio diffusers transformers accelerate

In [2]:
# Standard library imports
from io import BytesIO
import os

# Third-party library imports
import gradio as gr
from PIL import Image
import requests
from torch import autocast
import torch

# Machine learning model imports
from diffusers import StableDiffusionXLPipeline

## Biblioteki
* `BytesIO` to narzędzie, które pozwala nam pracować z danymi binarnymi w pamięci komputera - wyobraźmy to sobie jako wirtualny plik, który istnieje tylko w pamięci RAM. Jest to szczególnie przydatne gdy przetwarzamy obrazy, ponieważ nie musimy zapisywać ich tymczasowo na dysk.
* `Gradio` to narzędzie, które zamienia nasze modele uczenia maszynowego w przyjazne aplikacje webowe.
* `PIL` (Python Imaging Library) dostarcza nam narzędzia do manipulacji obrazami, podobnie jak program graficzny, ale sterowany kodem.
* `PyTorch` to framework do uczenia maszynowego, a autocast to mechanizm, który automatycznie optymalizuje wykorzystanie pamięci GPU podczas obliczeń.
* `StableDiffusionXLPipeline` z biblioteki diffusers. To najnowsza wersja modelu Stable Diffusion, zaprojektowana do generowania wysokiej jakości obrazów.
   * Pipeline w nazwie oznacza, że zawiera wszystkie potrzebne komponenty połączone w jeden sprawnie działający system - jak linia produkcyjna, gdzie tekst wchodzi na początku, a na końcu otrzymujemy gotowy obraz.

In [3]:
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [15]:
class CFG:
    model = "stabilityai/stable-diffusion-xl-base-1.0"
    device = 'cpu'
    dtype = torch.float16
    variant = "fp16"

if torch.backends.mps.is_available():
    CFG.device = "mps"
if torch.cuda.is_available():
    CFG.device = "cuda"

print("CUDA: ", torch.cuda.is_available())
print("MPS: ", torch.backends.mps.is_available())
print("Device: ", CFG.device)

CUDA:  True
MPS:  False
Device:  cuda


`model` wskazuje na konkretną wersję modelu Stable Diffusion XL - jest to "stabilityai/stable-diffusion-xl-base-1.0", czyli bazowy model SDXL w wersji 1.0 stworzony przez Stability AI. Ten model jest znany z generowania obrazów w wysokiej rozdzielczości.

`dtype` określa format danych jako torch.bfloat16 - jest to specjalny format liczb zmiennoprzecinkowych, który zapewnia dobry kompromis między precyzją obliczeń a zużyciem pamięci. Format bfloat16 wykorzystuje 16 bitów na liczbę, ale inaczej rozkłada bity między część całkowitą i ułamkową niż standardowy float16.
> Musiałem zmienic na float16 z racji błędu `expected mat1 and mat2 to have the same dtype`

`variant` ustawiony na "fp16" informuje, że model będzie działał w trybie połowicznej precyzji (16-bitowej). Jest to zgodne z wcześniejszym ustawieniem dtype i pozwala na szybsze obliczenia przy akceptowalnej utracie precyzji.

# Funkcje

In [5]:
def create_image(prompt):
    image = pipe(prompt=prompt).images[0]
    return image

# Aplikacja

In [9]:
pipe = StableDiffusionXLPipeline.from_pretrained(
      CFG.model,
      torch_dtype= CFG.dtype,
      variant = CFG.variant,
      use_safetensors=True).to(CFG.device)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

> Tworzenie zdjecia moze potrwac nawet z kilka minut na CPU. Na GPU powinno to trwać znacząco szybciej. 💪

In [10]:
demo = gr.Interface(fn=create_image, inputs="text", outputs="image")

demo.launch(debug = True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://58bc1841cd378ee71b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://58bc1841cd378ee71b.gradio.live
